In [ ]:
# Download Image data train+test
import boto3
from pathlib import Path
from botocore import UNSIGNED
from botocore.client import Config
from tqdm.notebook import tqdm

import zipfile
import os

def get_file_folders(s3_client, bucket_name, prefix=""):
    file_names = []
    folders = []

    default_kwargs = {
        "Bucket": bucket_name,
        "Prefix": prefix
    }
    next_token = ""

    while next_token is not None:
        updated_kwargs = default_kwargs.copy()
        if next_token != "":
            updated_kwargs["ContinuationToken"] = next_token

        response = s3_client.list_objects_v2(**updated_kwargs)
        contents = response.get("Contents")

        for result in contents:
            key = result.get("Key")
            if key[-1] == "/":
                folders.append(key)
            else:
                file_names.append(key)

        next_token = response.get("NextContinuationToken")

    return file_names, folders



def download_files(s3_client, bucket_name, local_path, file_names, folders):
    local_path = Path(local_path)

    for folder in tqdm(folders):
        folder_path = Path.joinpath(local_path, folder)
    # Create all folders in the path
        folder_path.mkdir(parents=True, exist_ok=True)

    for file_name in tqdm(file_names):
        file_path = Path.joinpath(local_path, file_name)
    # Create folder for parent directory
        file_path.parent.mkdir(parents=True, exist_ok=True)
        s3_client.download_file(
            bucket_name,
            file_name,
            str(file_path)
        )

        
client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
file_names, folders = get_file_folders(client, 'cgiar-crop-damage-classification-challenge')

download_files(
        client,
        'cgiar-crop-damage-classification-challenge',
        "C:\\Users\\yashr\\OneDrive\\Desktop\\minor_project",
        file_names,
        folders
    )



# Extract the contents of the zip file
with zipfile.ZipFile('C:\\Users\\yashr\\OneDrive\\Desktop\\minor_project\\images.zip', 'r') as zip_ref:
    zip_ref.extractall('C:\\Users\\yashr\\OneDrive\\Desktop\\minor_project\\images.zip')
    
    
# Check if the file exists before attempting to delete
if os.path.exists('C:\\Users\\yashr\\OneDrive\\Desktop\\minor_project\\images.zip'):
    # Delete the zip file
    os.remove('C:\\Users\\yashr\\OneDrive\\Desktop\\minor_project\\images.zip')
    print("The zip file has been deleted.")
else:
    print("The zip file does not exist.")
